### **IBM Data Science Peer Graded Assignment**
## **Part1**

## **1.DataFrame Creation**

In [2]:

!pip install geopy
!pip install beautifulsoup4

     |████████████████████████████████| 102kB 18.8MB/s ta 0:00:01
     |████████████████████████████████| 102kB 19.3MB/s ta 0:00:01


In [3]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries are imported.")

Libraries are imported.


## **2.Getting data from Wikipedia into a dataframe**

In [4]:
data_1 = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
# parse data from the html into a beautifulsoup object
Objsoup = BeautifulSoup(data_1, 'html.parser')

In [6]:
# creating three lists to store table data
postalCode = []
borough= []
neighborhood = []

**Using Beautifulsoup**

In [7]:
 #find the table
Objsoup.find('table').find_all('tr')

# find all the rows of the table
Objsoup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in Objsoup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [8]:
 #appending the data into the respective lists
for row in Objsoup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCode.append(cells[0].text)
        borough.append(cells[1].text)
        neighborhood.append(cells[2].text.rstrip('\n')) 

In [9]:
# creating a new DataFrame from the three new lists
Toronto_df = pd.DataFrame({"POSTALCODE": postalCode,
                           "BOROUGH": borough,
                           "NEIGHBORHOOD": neighborhood})

Toronto_df.head()

,POSTALCODE,BOROUGH,NEIGHBORHOOD
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


**Dropping "Not assigned"**

In [10]:
Toronto_df_new = Toronto_df[Toronto_df.BOROUGH != "Not assigned"].reset_index(drop=True)
Toronto_df_new.head(10)

,POSTALCODE,BOROUGH,NEIGHBORHOOD
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


**Grouping Neighborhood in the same borough**

In [11]:
Toronto_df_group = Toronto_df_new.groupby(["POSTALCODE", "BOROUGH"], as_index=False).agg(lambda x: ", ".join(x))
Toronto_df_group.head()

,POSTALCODE,BOROUGH,NEIGHBORHOOD
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Where neighborhood is"Not assigned" making the value same as Borough**

In [12]:
for index, row in Toronto_df_group.iterrows():
    if row["NEIGHBORHOOD"] == "Not assigned":
        row["NEIGHBORHOOD"] = row["BOROUGH"]
        
Toronto_df_group.head()

,POSTALCODE,BOROUGH,NEIGHBORHOOD
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Testing the requirements**

In [13]:
 #creating a new test dataframe
column_names = ["POSTALCODE", "BOROUGH", "NEIGHBORHOOD"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(Toronto_df_group[Toronto_df_group["POSTALCODE"]==postcode], ignore_index=True)
    
test_df

,POSTALCODE,BOROUGH,NEIGHBORHOOD
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


In [14]:

Toronto_df_group[Toronto_df_group['BOROUGH'] == 'Queen\'s Park']
Toronto_df_group

,POSTALCODE,BOROUGH,NEIGHBORHOOD
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [15]:
#shape of the dataframe
Toronto_df_group.shape

(103, 3)

### **IBM Data Science Peer Graded Assignment**
# **Part2**

**Loading the file from CSV File**

In [16]:

import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

link = "http://cocl.us/Geospatial_data"
df = pd.read_csv(link)

df.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df.rename(columns={"Postal Code": "POSTALCODE"}, inplace=True)
df.head()

,POSTALCODE,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merging two tables to attain the coordinates**

In [18]:
Toronto_df_new = Toronto_df_group.merge(df, on="POSTALCODE", how="left")
Toronto_df_new.head()

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Testing**

In [19]:
column_names = ["POSTALCODE", "BOROUGH", "NEIGHBORHOOD", "Latitude", "Longitude"]
test1_df = pd.DataFrame(columns=column_names)

test1_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test1_list:
    test1_df = test1_df.append(Toronto_df_new[Toronto_df_new["POSTALCODE"]==postcode], ignore_index=True)
    
test1_df

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


In [20]:
test1_df.to_csv(r'df_final.csv')

### **IBM Data Science Peer Graded Assignment**
**Part3**

**the latitude and longitude values of Toronto**

In [21]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


**Creating a map of Toronto with neighborhoods**

In [28]:

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, BOROUGH, NEIGHBORHOOD in zip(Toronto_df_new['Latitude'], Toronto_df_new['Longitude'], Toronto_df_new['BOROUGH'], Toronto_df_new['NEIGHBORHOOD']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
     parse_html=False).add_to(map_Toronto)  
    
map_Toronto

**Filtering the Boroughs that contains the word Toronto**

In [30]:

# filter borough names that contain the word Toronto
borough_names = list(Toronto_df_new.BOROUGH.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [31]:

# create a new DataFrame with only boroughs that contain the word Toronto
Toronto_df_new = Toronto_df_new[Toronto_df_new['BOROUGH'].isin(borough_with_toronto)].reset_index(drop=True)
print(Toronto_df_new.shape)
Toronto_df_new.head()

(38, 5)


,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [33]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_df_new['Latitude'], Toronto_df_new['Longitude'], Toronto_df_new['BOROUGH'], Toronto_df_new['NEIGHBORHOOD']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
     parse_html=False).add_to(map_Toronto) 
    
map_Toronto

 **Using the Foursquare API to explore the neighborhoods**

In [34]:
# define Foursquare Credentials and Version
CLIENT_ID = 'AJ5GIVG4RTCVSPB1CQBINDQIL435MQJ5X32O3DX2C2SL2NZM' # your Foursquare ID
CLIENT_SECRET = 'HT0QEMRESM552MVJNVZEX44HOYVPHNLYM0HZDRQJTH3MFIOQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AJ5GIVG4RTCVSPB1CQBINDQIL435MQJ5X32O3DX2C2SL2NZM
CLIENT_SECRET:HT0QEMRESM552MVJNVZEX44HOYVPHNLYM0HZDRQJTH3MFIOQ


**top 100 venues that are within a radius of 500 meters.**

In [35]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post,BOROUGH,NEIGHBORHOOD in zip(Toronto_df_new['Latitude'], Toronto_df_new['Longitude'], Toronto_df_new['POSTALCODE'], Toronto_df_new['BOROUGH'], Toronto_df_new['NEIGHBORHOOD']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            BOROUGH,
            NEIGHBORHOOD,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [36]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['POSTALCODE', 'BOROUGH', 'NEIGHBORHOOD', 'BOROUGHLATITUDE', 'BOROUGHLONGITUDE', 'VENUENAME', 'VENUELATITUDE', 'VENUELONGITUDE', 'VENUECATEGORY']

print(venues_df.shape)
venues_df.head()

(1689, 9)


,POSTALCODE,BOROUGH,NEIGHBORHOOD,BOROUGHLATITUDE,BOROUGHLONGITUDE,VENUENAME,VENUELATITUDE,VENUELONGITUDE,VENUECATEGORY
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


**Venues under each postalcode**

In [37]:
venues_df.groupby(["POSTALCODE","BOROUGH","NEIGHBORHOOD"]).count()

,,,BOROUGHLATITUDE,BOROUGHLONGITUDE,VENUENAME,VENUELATITUDE,VENUELONGITUDE,VENUECATEGORY
POSTALCODE,BOROUGH,NEIGHBORHOOD,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"The Beaches West, India Bazaar",20,20,20,20,20,20
M4M,East Toronto,Studio District,37,37,37,37,37,37
M4N,Central Toronto,Lawrence Park,5,5,5,5,5,5
M4P,Central Toronto,Davisville North,7,7,7,7,7,7
M4R,Central Toronto,North Toronto West,20,20,20,20,20,20
M4S,Central Toronto,Davisville,33,33,33,33,33,33
M4T,Central Toronto,"Moore Park, Summerhill East",3,3,3,3,3,3


In [39]:
print('These are the {} uniques categories.'.format(len(venues_df['VENUECATEGORY'].unique())))

These are the 233 uniques categories.


In [40]:
venues_df['VENUECATEGORY'].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Greek Restaurant', 'Cosmetics Shop', 'Italian Restaurant',
       'Ice Cream Shop', 'Brewery', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Pizza Place', 'Juice Bar', 'Bookstore',
       'Restaurant', 'Dessert Shop', 'Spa', 'Furniture / Home Store',
       'Diner', 'Bubble Tea Shop', 'Grocery Store', 'Coffee Shop',
       'Indian Restaurant', 'Caribbean Restaurant', 'Bakery',
       'American Restaurant', 'Liquor Store', 'Fish & Chips Shop', 'Gym',
       'Burger Joint', 'Park', 'Sushi Restaurant', 'Steakhouse',
       'Pet Store', 'Fast Food Restaurant', 'Burrito Place',
       'Movie Theater', 'Sandwich Place', 'Food & Drink Shop',
       'Fish Market', 'Café', 'Seafood Restaurant', 'Gay Bar',
       'Thai Restaurant', 'Middle Eastern Restaurant',
       'Comfort Food Restaurant', 'Cheese Shop', 'Stationery Store',
       'Coworking Space', 'Bar'], dtype=object)

## **  ANALYZING EACH AREA:**

In [43]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VENUECATEGORY']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['POSTALCODE'] = venues_df['POSTALCODE'] 
toronto_onehot['BOROUGH'] = venues_df['BOROUGH'] 
toronto_onehot['NEIGHBORHOODS'] = venues_df['NEIGHBORHOOD'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1689, 236)


,POSTALCODE,BOROUGH,NEIGHBORHOODS,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

**Now group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [44]:
toronto_grouped = toronto_onehot.groupby(["POSTALCODE", "BOROUGH", "NEIGHBORHOODS"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(38, 236)


,POSTALCODE,BOROUGH,NEIGHBORHOODS,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

**Creating new dataframe and display the top 10 venues for each PostalCode**

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['POSTALCODE', 'BOROUGH', 'NEIGHBORHOODS']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['POSTALCODE'] = toronto_grouped['POSTALCODE']
neighborhoods_venues_sorted['BOROUGH'] = toronto_grouped['BOROUGH']
neighborhoods_venues_sorted['NEIGHBORHOODS'] = toronto_grouped['NEIGHBORHOODS']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(38, 13)


,POSTALCODE,BOROUGH,NEIGHBORHOODS,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Trail,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Yoga Studio,Brewery,Bubble Tea Shop,Pub
2,M4L,East Toronto,"The Beaches West, India Bazaar",Sandwich Place,Coffee Shop,Ice Cream Shop,Burger Joint,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Sandwich Place,Fish Market,Bookstore,Coworking Space,Latin American Restaurant
4,M4N,Central Toronto,Lawrence Park,Photography Studio,Park,Lawyer,Bus Line,Swim School,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store
5,M4P,Central Toronto,Davisville North,Food & Drink Shop,Gym,Park,Breakfast Spot,Sandwich Place,Clothing Store,Hotel,Yoga Studio,Dog Run,Doner Restaurant
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Sporting Goods Shop,Clothing Store,Yoga Studio,Diner,Dessert Shop,Mexican Restaurant,Salon / Barbershop,Metro Station,Ice Cream Shop
7,M4S,Central Toronto,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Gym,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Greek Restaurant,Fried Chicken Joint
8,M4T,Central Toronto,"Moore Park, Summerhill East",Gym,Trail,Restaurant,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,Pizza Place,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Bagel Shop,Liquor Store,Fried Chicken Joint


**Using K means to form the cluster**

In [46]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["POSTALCODE","BOROUGH","NEIGHBORHOODS"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 3, 3, 2, 3, 3, 3, 3, 3], dtype=int32)

In [47]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = Toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["BOROUGH", "NEIGHBORHOODS"], 1).set_index("POSTALCODE"), on="POSTALCODE")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!


(38, 16)


,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Yoga Studio,Brewery,Bubble Tea Shop,Pub
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3,Sandwich Place,Coffee Shop,Ice Cream Shop,Burger Joint,Liquor Store,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Sandwich Place,Fish Market,Bookstore,Coworking Space,Latin American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Photography Studio,Park,Lawyer,Bus Line,Swim School,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store


In [48]:
##sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(38, 16)


,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Home Service,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Photography Studio,Park,Lawyer,Bus Line,Swim School,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,3,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,American Restaurant,Bakery,Steakhouse,Gym,Seafood Restaurant
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3,Sandwich Place,Café,Coffee Shop,American Restaurant,Burger Joint,Pub,Indian Restaurant,Pizza Place,BBQ Joint,History Museum
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,3,Café,Japanese Restaurant,Restaurant,Sandwich Place,Bookstore,Bakery,Bar,Italian Restaurant,Noodle House,Dessert Shop
26,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,3,Café,Bar,Vietnamese Restaurant,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Bakery,Donut Shop
27,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,3,Airport Lounge,Airport Service,Bar,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Airport Terminal,Boat or Ferry,Plane
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,3,Coffee Shop,Restaurant,Café,Hotel,Beer Bar,Italian Restaurant,Bakery,Japanese Restaurant,Seafood Restaurant,Creperie
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,3,Coffee Shop,Café,Hotel,Steakhouse,Seafood Restaurant,Gym,Bakery,Bar,American Restaurant,Restaurant


## **Visualize the Clusters**

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['POSTALCODE'], toronto_merged['BOROUGH'], toronto_merged['NEIGHBORHOOD'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Cluster1**

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,BOROUGH,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Trail,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


**Cluster2**

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,BOROUGH,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Garden,Home Service,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


**Cluster3**

In [53]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,BOROUGH,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Photography Studio,Park,Lawyer,Bus Line,Swim School,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store


**Cluster4**

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,BOROUGH,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,American Restaurant,Bakery,Steakhouse,Gym,Seafood Restaurant
24,Central Toronto,3,Sandwich Place,Café,Coffee Shop,American Restaurant,Burger Joint,Pub,Indian Restaurant,Pizza Place,BBQ Joint,History Museum
25,Downtown Toronto,3,Café,Japanese Restaurant,Restaurant,Sandwich Place,Bookstore,Bakery,Bar,Italian Restaurant,Noodle House,Dessert Shop
26,Downtown Toronto,3,Café,Bar,Vietnamese Restaurant,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Bakery,Donut Shop
27,Downtown Toronto,3,Airport Lounge,Airport Service,Bar,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Airport Terminal,Boat or Ferry,Plane
28,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Hotel,Beer Bar,Italian Restaurant,Bakery,Japanese Restaurant,Seafood Restaurant,Creperie
29,Downtown Toronto,3,Coffee Shop,Café,Hotel,Steakhouse,Seafood Restaurant,Gym,Bakery,Bar,American Restaurant,Restaurant
30,Downtown Toronto,3,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Baby Store,Restaurant,Candy Store,Convenience Store
31,West Toronto,3,Pharmacy,Supermarket,Bakery,Park,Gym / Fitness Center,Café,Music Venue,Middle Eastern Restaurant,Bar,Brewery
32,West Toronto,3,Bar,Coffee Shop,Men's Store,Restaurant,Asian Restaurant,New American Restaurant,Vietnamese Restaurant,Café,French Restaurant,Bakery


**Cluster5**

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,BOROUGH,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
10,Downtown Toronto,4,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### **Inference:**

## **Most of the neighborhoods fall into Cluster 4 which are mostly business areas with cafe,coffee shops etc. Cluster 2 is just a garden, Cluster 3 has Photography studio, Cluster 1 has Health food studio and lastly Cluster 5 has park and trail.**